In [2]:
import os

SEASONS = list(range(2020,2025))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings") #standings directory
SCORES_DIR = os.path.join(DATA_DIR, "scores") #scores directory

In [3]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [4]:
#get link
async def get_html(url, selector, sleep=1, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [5]:
#scrape all months of a season 
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
!playwright install


Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libopus.so.0                                     ║
║     libwebpdemux.so.2                                ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libhyphen.so.0                                   ║
║     libflite.so.1                                    ║
║     libflite_usenglish.so.1                          ║
║     libflite_cmu_grapheme_lang.so.1                  ║
║     libflite_cmu_grapheme_lex.so.1                   ║
║     libflite_cmu_indic_lang.so.1                     ║
║     libflite_cmu_indic_lex.so.1                      ║
║     libflite_cmulex.so.1                             ║
║     libflite_cmu_time_awb.so.1                   

In [7]:
await scrape_season(2021)

2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com


In [8]:
standings_files = os.listdir(STANDINGS_DIR)

In [11]:
#scrape games from html files in standings 
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [13]:
import pandas as pd
#for season in SEASONS:
files = [s for s in standings_files if str(2021) in s]
    
for f in files:
    filepath = os.path.join(STANDINGS_DIR, f)
        
    await scrape_game(filepath)

Pistons vs Hornets, May 1, 2021 | Basketball-Reference.com
Warriors vs Rockets, May 1, 2021 | Basketball-Reference.com
Bulls vs Hawks, May 1, 2021 | Basketball-Reference.com
Heat vs Cavaliers, May 1, 2021 | Basketball-Reference.com
Pelicans vs Timberwolves, May 1, 2021 | Basketball-Reference.com
Pacers vs Thunder, May 1, 2021 | Basketball-Reference.com
Grizzlies vs Magic, May 1, 2021 | Basketball-Reference.com
Wizards vs Mavericks, May 1, 2021 | Basketball-Reference.com
Nuggets vs Clippers, May 1, 2021 | Basketball-Reference.com
Raptors vs Jazz, May 1, 2021 | Basketball-Reference.com
Nets vs Bucks, May 2, 2021 | Basketball-Reference.com
Trail Blazers vs Celtics, May 2, 2021 | Basketball-Reference.com
Heat vs Hornets, May 2, 2021 | Basketball-Reference.com
Kings vs Mavericks, May 2, 2021 | Basketball-Reference.com
Knicks vs Rockets, May 2, 2021 | Basketball-Reference.com
Suns vs Thunder, May 2, 2021 | Basketball-Reference.com
76ers vs Spurs, May 2, 2021 | Basketball-Reference.com
Raptor

CancelledError: 